In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [2]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [3]:
%matplotlib widget

In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
## force template
dist = 13.39
z_sep = -15.85
y_offset = 4.9
G_lambda = 20
test = force_vs_time(dist,z_sep,-200,3,G_lambda,"z", yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = np.sqrt(2)*fft_norm(50000, 5000)

_,ax = plt.subplots()
x = list(test[1])*10
fft1 = np.abs(np.fft.rfft(x))*scale/np.sqrt(10)
fft_angles = np.angle(np.fft.rfft(x))
freq1 = np.fft.rfftfreq(len(x), d=1./5000)
ax.semilogy(freq1, fft1, label='7.6 $\mu m$ sphere - Z=-15.7')
ax.set(xlim=(0,30), ylim=(None,None), xlabel='frequency [Hz]', ylabel='Force [N]', title=r'$\alpha=1x10^{8}$')
ax.legend()
print(fft1[freq1==12])

Loading Gravity Data... Done!
68 2.04837466632e-05
Loaded Yukawa Force


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[  5.42043498e-18]


In [24]:
[print(fft1[freq1==freq_][0]) for freq_ in 3*np.arange(1,14)]

4.68952985727e-18
6.86879411916e-18
2.92835872514e-18
8.61637877166e-18
1.48985916779e-18
7.15587765636e-18
6.99119057336e-18
2.49970446358e-18
4.412814934e-18
1.03432982087e-17
8.87405984508e-18
8.90155956221e-18
4.82608802896e-18


[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [23]:
def func_psd(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Discharge/New_Discharge/Discharge_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking373/Shaking4_'+str(i)+'.h5'

    bb = BeadDataFile.BeadDataFile(fname=fname)
    fft_ = bb.psd2('z')[1]
    return (fft_)

In [24]:
test_norms = np.array(Parallel(n_jobs=20)(delayed(func_psd)(i) for i in tqdm(range(1000))))

100%|██████████| 1000/1000 [00:11<00:00, 79.37it/s]


In [25]:
i = 100
fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
# fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking373/Shaking4_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
dist = 392.4-np.mean(bb.cant_pos[0])*50-7.6/2
freq_psd = bb.psd2('z')[0]
shaking3_psd = np.mean(test_norms3, axis=0)
noshaking1_psd = np.mean(test_norms, axis=0)
# shaking4_psd = np.mean(test_norms3, axis=0)

In [8]:
z_scale2

array([  3.99023794e-20])

In [11]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/New_Discharge/',
                         start_file=0, max_file=1000)
bdf_z = bdf_discharge[:5]

freq = bdf_z[0].psd2('z')[0]
z_scale2 = bdf_z[0].psd2('z')[1][freq==71]
for bb in bdf_z:
    z_scale2 += bb.psd2('z')[1][freq==71]
z_scale2 /= len(bdf_z)

charges = 7
force = charges * 1.6e-19 * 20 / 8.6e-3 * 0.63 # in Newtons
z_scale2 = force/np.sqrt(z_scale2)

No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
15  files in folder
15  files loaded


In [85]:
shaking3_psd.shape, correction.shape

((25001,), (25001,))

In [34]:
_,ax = plt.subplots(figsize=(9,4))
ax.semilogy(freq_psd, np.sqrt(shaking3_psd)*z_scale2, label='Wilson 378-3')
# ax.semilogy(freq_psd, 0.9*np.sqrt(shaking4_psd)*z_scale2, label='Wilson 373-4')
ax.semilogy(freq_psd, 1.13*np.sqrt(noshaking1_psd)*z_scale2+fft1*np.sqrt(correction), '--', label=r'noise+signal[$\alpha=10^8$]', alpha=0.8)
# ax.semilogy(freq_psd, 1.03*np.sqrt(noshaking1_psd)*z_scale2, 'r--', label=r'noise only', alpha=0.8)
ax.legend()
_ = ax.set(xlim=(50,87), ylim=(1e-17,2e-16), xlabel='Frequency [Hz.]', ylabel='Force [N]', title=r'Z force')
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
fft_tf[freq_tf==2]

array([ 0.02537228])

In [16]:
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
_,ax = plt.subplots(figsize=(9,4))
freq_tf, fft_tf = tf_z.psd2('z')
fft_tf /= fft_tf[freq_tf==71]
freq_corr = np.arange(1,100)
value_corr = [fft_tf[freq_tf==freq_] for freq_ in freq_corr]
correction = np.interp(freq_psd, freq_corr, np.array(value_corr).flatten())
ax.semilogy(freq_tf, np.sqrt(fft_tf))
ax.semilogy(freq_psd, np.sqrt(correction), '--')
_ = ax.set(xlim=(1,100), xlabel='Frequency [Hz.]', ylabel='Response [a.u]', title=r'Z force - TF')

No height data
No cantilever data
Error loading spinning data
No laser power data


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Phase

In [36]:
drive_freq = 9

def func_filter(i):
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Discharge/New_Discharge/Discharge_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking373/Shaking4_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200614/Bead1/Shaking/Shaking367/SpinShaking4_'+str(i)+'.h5'


    bb = BeadDataFile.BeadDataFile(fname=fname)
    res = bb.response_at_freq2('z', drive_freq=drive_freq, bandwidth=2)
    return (res)

In [37]:
test_phases = np.array(Parallel(n_jobs=40)(delayed(func_filter)(i) for i in tqdm(range(4000))))



  0%|          | 0/4000 [00:00<?, ?it/s]

  1%|          | 40/4000 [00:03<05:53, 11.22it/s]

  2%|▏         | 80/4000 [00:04<04:44, 13.78it/s]

  3%|▎         | 120/4000 [00:05<03:45, 17.21it/s]

  4%|▍         | 160/4000 [00:06<02:57, 21.63it/s]

  5%|▌         | 200/4000 [00:07<02:26, 25.93it/s]

  6%|▌         | 240/4000 [00:08<02:01, 30.88it/s]

  7%|▋         | 280/4000 [00:08<01:43, 35.82it/s]

  8%|▊         | 320/4000 [00:09<01:31, 40.24it/s]

  9%|▉         | 360/4000 [00:10<01:34, 38.58it/s]

 10%|█         | 400/4000 [00:11<01:23, 43.18it/s]

 11%|█         | 440/4000 [00:12<01:17, 46.09it/s]

 12%|█▏        | 480/4000 [00:12<01:15, 46.61it/s]

 13%|█▎        | 520/4000 [00:13<01:09, 50.38it/s]

 14%|█▍        | 560/4000 [00:14<01:04, 53.13it/s]

 15%|█▌        | 600/4000 [00:15<01:05, 52.10it/s]

 16%|█▌        | 640/4000 [00:15<01:04, 52.35it/s]

 17%|█▋        | 680/4000 [00:16<01:02, 53.46it/s]

 18%|█▊        | 720/4000 [00:17<01:01, 53.38it/s]

 19%|█▉        | 760/4

In [38]:
x = list(test[1])*10
fft_angles = np.angle(np.fft.rfft(x))
freq1 = np.fft.rfftfreq(len(x), d=1./5000)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

No height data
No cantilever data
Error loading spinning data
No laser power data


In [39]:
phase_th = fft_angles[freq1==drive_freq]  ## theory 
phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)

In [40]:
ll = likelihood_analyser.LikelihoodAnalyser()
phases = []
amps = []
fit_kwargs = {'A': 1, 'f':drive_freq, 'phi': np.pi,
          'error_A': 1, 'error_phi': 0.5, 'error_f': 1, 'errordef': 1,
          'limit_phi': [0, 2 * np.pi],
          'limit_A': [-100000, 100000],
          'print_level': 0, 'fix_A': False, 'fix_phi': False, 'fix_f': True}
for i in range(400):
    tmp_avg = np.mean(test_phases[10*i:(i+1)*10], axis=0)
    mm = ll.find_mle_sin(tmp_avg[5000:-5000]*10000, **fit_kwargs)
    print(mm.values)
    phases.append(mm.values[2])
    amps.append(mm.values[0])

<ValueView of Minuit at 9af9cf8>
  A: 56815.29564618878
  f: 9.0
  phi: 2.726458477780551
<ValueView of Minuit at 4c89258>
  A: 45632.338256796706
  f: 9.0
  phi: 2.5622717034155436
<ValueView of Minuit at a22ad28>
  A: 66385.76909253694
  f: 9.0
  phi: 3.7112902913587695
<ValueView of Minuit at 100fe908>
  A: 100000.0
  f: 9.0
  phi: 3.690355014076487
<ValueView of Minuit at 100fc9c8>
  A: 64838.82145823017
  f: 9.0
  phi: 3.651114331485412
<ValueView of Minuit at 49f11a8>
  A: 95391.33617240275
  f: 9.0
  phi: 3.878851450649892
<ValueView of Minuit at 4cc3448>
  A: 50248.97526882711
  f: 9.0
  phi: 2.581218074766646
<ValueView of Minuit at 4a11b18>
  A: 100000.0
  f: 9.0
  phi: 1.0242565305154698
<ValueView of Minuit at 9bd2de8>
  A: 100000.0
  f: 9.0
  phi: 0.7734402505115252
<ValueView of Minuit at 47eecb8>
  A: 78947.92905673222
  f: 9.0
  phi: 3.6667530886141866
<ValueView of Minuit at 49fb798>
  A: 99413.63726623557
  f: 9.0
  phi: 3.716730906592613
<ValueView of Minuit at 5519d

<ValueView of Minuit at a5357b8>
  A: 100000.0
  f: 9.0
  phi: 3.0095449467822664
<ValueView of Minuit at a597138>
  A: 100000.0
  f: 9.0
  phi: 3.532058583031546
<ValueView of Minuit at 9af9cf8>
  A: 99999.99999999997
  f: 9.0
  phi: 3.5515001352537636
<ValueView of Minuit at 4c89258>
  A: -42693.15832554047
  f: 9.0
  phi: 4.208003910917527
<ValueView of Minuit at a22ad28>
  A: 100000.0
  f: 9.0
  phi: 0.7987790357091663
<ValueView of Minuit at 100fe908>
  A: 55961.27019387024
  f: 9.0
  phi: 3.7412827026839026
<ValueView of Minuit at 100fc9c8>
  A: 100000.0
  f: 9.0
  phi: 3.756798368972956
<ValueView of Minuit at 49f11a8>
  A: 74803.43903581903
  f: 9.0
  phi: 3.862180248641007
<ValueView of Minuit at 4cc3448>
  A: 100000.0
  f: 9.0
  phi: 3.5115319244424725
<ValueView of Minuit at 4a11b18>
  A: 100000.0
  f: 9.0
  phi: 4.199091643100564
<ValueView of Minuit at 9bd2de8>
  A: 75424.96423756317
  f: 9.0
  phi: 3.9248092314183287
<ValueView of Minuit at 47eecb8>
  A: 100000.0
  f: 9.0

<ValueView of Minuit at 4125db8>
  A: 58902.1337321265
  f: 9.0
  phi: 3.894538906707531
<ValueView of Minuit at 4119818>
  A: 86120.56764667798
  f: 9.0
  phi: 3.4703875227043173
<ValueView of Minuit at a116358>
  A: 94004.11380002156
  f: 9.0
  phi: 3.7426218310186674
<ValueView of Minuit at 48e42c8>
  A: 100000.0
  f: 9.0
  phi: 3.284800479529386
<ValueView of Minuit at 9c0d0e8>
  A: 70358.05756424688
  f: 9.0
  phi: 3.239112985662461
<ValueView of Minuit at a10b1f8>
  A: 77557.26088225661
  f: 9.0
  phi: 2.707332637849401
<ValueView of Minuit at 9ad2238>
  A: 69268.1468637122
  f: 9.0
  phi: 3.6599043766684503
<ValueView of Minuit at a58eee8>
  A: 66494.05282406506
  f: 9.0
  phi: 3.7733730858254044
<ValueView of Minuit at 558f618>
  A: 87041.47378987039
  f: 9.0
  phi: 3.5799499129327983
<ValueView of Minuit at 554a568>
  A: 64606.00311738928
  f: 9.0
  phi: 2.140539067504081
<ValueView of Minuit at 9ac75f8>
  A: 61164.796794679685
  f: 9.0
  phi: 3.3000182626047794
<ValueView of 

<ValueView of Minuit at 49fb798>
  A: 60169.17954418916
  f: 9.0
  phi: 4.0152303351497265
<ValueView of Minuit at 47eecb8>
  A: 42745.48224249628
  f: 9.0
  phi: 3.1320137083865087
<ValueView of Minuit at 9bd2de8>
  A: 57539.55527565145
  f: 9.0
  phi: 3.126952897589735
<ValueView of Minuit at 551b0b8>
  A: 43621.43170471332
  f: 9.0
  phi: 3.171161576642988
<ValueView of Minuit at 4885bf8>
  A: 56318.9631368792
  f: 9.0
  phi: 3.867550324665497
<ValueView of Minuit at 478b148>
  A: 99999.99999999761
  f: 9.0
  phi: 3.3956740874881777
<ValueView of Minuit at 4722eb8>
  A: 96644.93463449855
  f: 9.0
  phi: 3.539832336257244
<ValueView of Minuit at 9adcbf8>
  A: 73421.84765331334
  f: 9.0
  phi: 3.182961031534656
<ValueView of Minuit at 55d4a18>
  A: 24459.461830730535
  f: 9.0
  phi: 2.9250178776642497
<ValueView of Minuit at 55531a8>
  A: 23073.593936136356
  f: 9.0
  phi: 4.097104443098646
<ValueView of Minuit at 4117d28>
  A: 16479.14841932492
  f: 9.0
  phi: 4.18141573896473
<Value

<ValueView of Minuit at 554a698>
  A: 66530.8308586611
  f: 9.0
  phi: 3.1563168995885205
<ValueView of Minuit at 4c27658>
  A: 32107.307924416993
  f: 9.0
  phi: 3.7997417540211513
<ValueView of Minuit at 552f898>
  A: 44313.26270259937
  f: 9.0
  phi: 3.119993304038345
<ValueView of Minuit at 4cb1e68>
  A: 42226.86029073587
  f: 9.0
  phi: 2.926732076718643
<ValueView of Minuit at 493bea8>
  A: 72582.49157567276
  f: 9.0
  phi: 3.0812883296907954
<ValueView of Minuit at 5555b68>
  A: 19007.080347200288
  f: 9.0
  phi: 4.798522291088301
<ValueView of Minuit at 4127228>
  A: 57927.52807171582
  f: 9.0
  phi: 3.6180880759056038
<ValueView of Minuit at 9bd3d28>
  A: 79547.89688259721
  f: 9.0
  phi: 3.3396616176316463
<ValueView of Minuit at a595aa8>
  A: 87633.11037243862
  f: 9.0
  phi: 3.4541605378694498
<ValueView of Minuit at a232f08>
  A: 76576.4709070737
  f: 9.0
  phi: 1.7868246226449727
<ValueView of Minuit at 4c78d68>
  A: 71828.78364997759
  f: 9.0
  phi: 3.265972707449847
<Va

In [33]:
_,ax = plt.subplots(figsize=(9,4.5))
ax.scatter(np.arange(0,4000,10)*10/60, phases, label='measured phase')
ax.plot(np.arange(0,4000,10)*10/60, np.arange(400)*0+phase_all, 'r--', label='theory-pull')
ax.plot(np.arange(0,4000,10)*10/60, np.arange(400)*0+phase_all+np.pi, 'g--', label='theory-push')
ax.legend()
ax.set(ylabel='Phase', xlabel='Time [minute]', title=str(drive_freq)+' Hz.')

ax.grid(True)
ax.yaxis.set_major_locator(plt.MultipleLocator(np.pi / 2))
ax.yaxis.set_minor_locator(plt.MultipleLocator(np.pi / 12))
ax.yaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [211]:
bandwidth = 2
fsamp = 5000
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
responsefilt = signal.filtfilt(b, a, x)
mm = ll.find_mle_sin(responsefilt[5000:-5000], **fit_kwargs)
phase_th = mm.values[2]

In [12]:
import numpy as np
import matplotlib.pyplot as plt

def multiple_formatter(denominator=2, number=np.pi, latex='\pi'):
    def gcd(a, b):
        while b:
            a, b = b, a%b
        return a
    def _multiple_formatter(x, pos):
        den = denominator
        num = np.int(np.rint(den*x/number))
        com = gcd(num,den)
        (num,den) = (int(num/com),int(den/com))
        if den==1:
            if num==0:
                return r'$0$'
            if num==1:
                return r'$%s$'%latex
            elif num==-1:
                return r'$-%s$'%latex
            else:
                return r'$%s%s$'%(num,latex)
        else:
            if num==1:
                return r'$\frac{%s}{%s}$'%(latex,den)
            elif num==-1:
                return r'$\frac{-%s}{%s}$'%(latex,den)
            else:
                return r'$\frac{%s%s}{%s}$'%(num,latex,den)
    return _multiple_formatter

class Multiple:
    def __init__(self, denominator=2, number=np.pi, latex='\pi'):
        self.denominator = denominator
        self.number = number
        self.latex = latex

    def locator(self):
        return plt.MultipleLocator(self.number / self.denominator)

    def formatter(self):
        return plt.FuncFormatter(multiple_formatter(self.denominator, self.number, self.latex))